In [128]:
import requests
from bs4 import BeautifulSoup
import time

base_url = "https://link.springer.com"
search_url = base_url + "/search/page/{}?facet-discipline=%22Economics%22&facet-content-type=%22Journal%22&facet-language=%22En%22"

Journal_list = []

# Iterate over pages from 1 to 6
for page_number in range(1, 7):
    url = search_url.format(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    journal_links = soup.find_all("a", class_="title")
    for link in journal_links:
        journal_url = link.get("href")
        journal_number = journal_url.split("/journal/")[-1]
        Journal_list.append(journal_number)

    # Sleep for 1 second before making the next request
    time.sleep(1)

# Print the collected journal numbers with an index
for index, number in enumerate(Journal_list, 1):
    print(f"{index}. {number}")


1. 12232
2. 11150
3. 10058
4. 10887
5. 712
6. 41685
7. 11403
8. 43253
9. 10614
10. 13132
11. 11166
12. 181
13. 191
14. 11294
15. 12076
16. 182
17. 11079
18. 11293
19. 12197
20. 10824
21. 355
22. 10657
23. 10640
24. 11116
25. 199
26. 11146
27. 12651
28. 40953
29. 40847
30. 13209
31. 40888
32. 41775
33. 41308
34. 11127
35. 148
36. 11123
37. 11698
38. 10037
39. 10797
40. 10888
41. 11369
42. 10203
43. 10109
44. 11507
45. 10818
46. 11238
47. 40854
48. 10663
49. 41027
50. 10644
51. 10290
52. 10368
53. 40100
54. 41885
55. 40797
56. 41294
57. 40812
58. 10645
59. 10258
60. 40505
61. 11151
62. 168
63. 40881
64. 11067
65. 10842
66. 41937
67. 43071
68. 10018
69. 41996
70. 40008
71. 13563
72. 11149
73. 42495
74. 41549
75. 40844
76. 42973
77. 10683
78. 550
79. 11138
80. 41302
81. 40822
82. 10101
83. 11299
84. 41130
85. 13393
86. 41412
87. 10272
88. 11066
89. 40503
90. 40176
91. 40173
92. 40604
93. 40172
94. 12114
95. 40589
96. 40552
97. 40175
98. 40174
99. 40932
100. 11300
101. 41318
102. 11459
103.

['12232', '11150', '10058', '10887', '712', '41685', '11403', '43253', '10614', '13132', '11166', '181', '191', '11294', '12076', '182', '11079', '11293', '12197', '10824', '355', '10657', '10640', '11116', '199', '11146', '12651', '40953', '40847', '13209', '40888', '41775', '41308', '11127', '148', '11123', '11698', '10037', '10797', '10888', '11369', '10203', '10109', '11507', '10818', '11238', '40854', '10663', '41027', '10644', '10290', '10368', '40100', '41885', '40797', '41294', '40812', '10645', '10258', '40505', '11151', '168', '40881', '11067', '10842', '41937', '43071', '10018', '41996', '40008', '13563', '11149', '42495', '41549', '40844', '42973', '10683', '550', '11138', '41302', '40822', '10101', '11299', '41130', '13393', '41412', '10272', '11066', '40503', '40176', '40173', '40604', '40172', '12114', '40589', '40552', '40175', '40174', '40932', '11300', '41318', '11459', '12143', '10108', '12113', '10160', '10110', '10829', '11039']


In [130]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gender_guesser.detector as gender
import time
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures


path_to_save = "/Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer/Journals_csv"



headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialize gender detector
detector = gender.Detector()

def get_first_name(full_name):
    if full_name:
        return full_name.split()[0]
    return "None"

def predict_gender_first_name(name):
    return detector.get_gender(name) if name != "None" else "unknown"

def fetch_soup(session, url):
    response = session.get(url, headers=headers)
    # Check for "Too Many Requests" and pause if encountered
    if response.status_code == 429:
        print("429 Too Many Requests encountered. Pausing for 10 minutes...")
        time.sleep(600)  # Sleep for 10 minutes
        return fetch_soup(session, url)  # Retry after pausing
    time.sleep(1)  # Sleep for 1 second between requests
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    return None


MAX_AUTHORS = 10

def process_journal(journal_number):
    data = {
        'Volume': [],
        'Issue': [],
        'Journal Name': [],
        'Published Date': [],
        'Link': [],
        'Title': [],
        'Journal Year': []
    }
    
    for i in range(1, MAX_AUTHORS+1):
        data[f'Author {i}'] = []

    print(f"Scraping data for journal number {journal_number}...")

    with requests.Session() as session:
        base_url = f'https://link.springer.com/journal/{journal_number}/volumes-and-issues/'
        volume = 1

        while True:
            print(f"Scraping volume {volume}...")

            issue = 1
            while True:
                url = base_url + f'{volume}-{issue}'
                soup = fetch_soup(session, url)

                if soup:
                    articles = soup.find_all('li', class_='c-list-group__item')
                    if not articles:
                        break

                    journal_tag = soup.find('div', {'id': 'journalTitle'}).find('a')
                    journal_name = journal_tag.text.strip() if journal_tag else None
                    journal_year = soup.find('h1', class_='u-mb-8')
                    journal_year = journal_year.text.split(",")[-1].strip() if journal_year else None

                    for article in articles:
                        link = article.find('a', href=True)['href']
                        authors = [author.text.strip() for author in article.select('ul.c-author-list li span')]
                        title = article.find('a', attrs={"data-track": "click"}).text.strip()
                        published_date_tag = article.find('li', attrs={'data-test': 'published-on'})
                        published_date = published_date_tag.text.split(': ')[1].strip() if published_date_tag else None

                        data['Volume'].append(volume)
                        data['Issue'].append(issue)
                        data['Journal Name'].append(journal_name)
                        data['Published Date'].append(published_date)
                        data['Link'].append(link)
                        data['Title'].append(title)
                        data['Journal Year'].append(journal_year)

                        # Fill in authors or None values
                        for i in range(MAX_AUTHORS):
                            if i < len(authors):
                                data[f'Author {i+1}'].append(authors[i])
                            else:
                                data[f'Author {i+1}'].append(None)

                else:
                    break

                issue += 1

            time.sleep(2)  # Sleep for 2 seconds after processing each volume
            next_soup = fetch_soup(session, base_url + f'{volume+1}-1')
            if not next_soup or (next_soup and not next_soup.find_all('li', class_='c-list-group__item')):
                break

            volume += 1

    df = pd.DataFrame(data)

    for i in range(MAX_AUTHORS):
        col_name = f'Author {i+1}'
        df[f'Gender_{col_name}'] = df[col_name].apply(get_first_name).apply(predict_gender_first_name)
        df[f'Gender_{col_name}'] = df[f'Gender_{col_name}'].map({
            'male': 'Male',
            'female': 'Female',
            'unknown': 'Unknown',
            'None': 'Unknown'
        })

    return df

# Dictionary to hold DataFrames for each journal
dfs = {}

import os

if not os.path.exists(path_to_save):
    os.makedirs(path_to_save)


# Use ThreadPoolExecutor to scrape multiple journals concurrently with limited threads
MAX_THREADS = 4
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    futures = {executor.submit(process_journal, journal): journal for journal in Journal_list}
    for future in concurrent.futures.as_completed(futures):
        journal_num = futures[future]
        try:
            result = future.result()
            dfs[journal_num] = pd.DataFrame(result)
            # Save the DataFrame to a CSV file in the specified directory
            dfs[journal_num].to_csv(f"{path_to_save}/journal_{journal_num}.csv", index=False)
            print(f"Saved CSV for journal {journal_num}")
        except Exception as e:
            print(f"Error processing journal {journal_num}: {e}")



# Print out the saved DataFrames
for journal, df in dfs.items():
    print(f"DataFrame for journal {journal}:\n")
    print(df)
    print("\n" + "="*50 + "\n")


Scraping data for journal number 12232...Scraping data for journal number 11150...
Scraping volume 1...

Scraping volume 1...
Scraping data for journal number 10058...
Scraping volume 1...
Scraping data for journal number 10887...
Scraping volume 1...
Scraping data for journal number 712...
Scraping volume 1...
Saved CSV for journal 12232
Scraping volume 2...
Scraping volume 2...
Scraping volume 2...
Scraping volume 3...
Scraping volume 3...
Scraping volume 2...
Scraping volume 3...
Scraping volume 4...
Scraping volume 4...
Scraping volume 3...
Scraping volume 4...
Scraping volume 5...
Scraping volume 5...
Scraping volume 5...
Scraping volume 4...
Scraping volume 6...
Scraping volume 6...
Scraping volume 6...
Scraping volume 7...
Scraping volume 5...
Scraping volume 7...
Scraping volume 7...
Scraping volume 8...
Scraping volume 6...
Scraping volume 8...
Scraping volume 8...
Scraping volume 9...
Scraping volume 7...
Scraping volume 9...
Scraping volume 9...
Scraping volume 10...
Scrapin

In [133]:
all_data = pd.concat(dfs.values(), ignore_index=True)

# Drop columns where all observations are empty and their corresponding Gender_Author columns
for i in range(1, MAX_AUTHORS + 1):
    author_col = f'Author {i}'
    gender_col = f'Gender_{author_col}'
    if all_data[author_col].isna().all():  # Check if all values in the column are NaN
        all_data.drop(columns=[author_col, gender_col], inplace=True)

# Save the combined dataframe to a CSV file
all_data.to_csv(f"{path_to_save}/all_journals.csv", index=False)
print("Saved all data to all_journals.csv")

# Print out the combined dataframe
print(all_data)
print("\n" + "="*50 + "\n")

Saved all data to all_journals.csv
       Volume  Issue                          Journal Name    Published Date  \
0         1.0    1.0  Review of Economics of the Household              None   
1         1.0    1.0  Review of Economics of the Household              None   
2         1.0    1.0  Review of Economics of the Household              None   
3         1.0    1.0  Review of Economics of the Household              None   
4         1.0    1.0  Review of Economics of the Household              None   
...       ...    ...                                   ...               ...   
80001    57.0    6.0                        Intereconomics  16 December 2022   
80002    57.0    6.0                        Intereconomics  16 December 2022   
80003    57.0    6.0                        Intereconomics  16 December 2022   
80004    57.0    6.0                        Intereconomics  16 December 2022   
80005    57.0    6.0                        Intereconomics  16 December 2022   

    

In [132]:
import os

path = "/Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer/Journals_csv"
csv_files = [f for f in os.listdir(path) if f.startswith("journal_") and f.endswith(".csv")]

for index, csv_file in enumerate(csv_files):
    # Extract the journal number from the filename
    journal_num = csv_file.replace("journal_", "").replace(".csv", "")
    print(f"{index + 1}. {journal_num}")


1. 11116
2. 11300
3. 40173
4. 10368
5. 40589
6. 40953
7. 41318
8. 11507
9. 40172
10. 10037
11. 11698
12. 11459
13. 41130
14. 12232
15. 10829
16. 11138
17. 40175
18. 10018
19. 41294
20. 11299
21. 41685
22. 43253
23. 12197
24. 40174
25. 40604
26. 40176
27. 41308
28. 11067
29. 11066
30. 40822
31. 10797
32. 12143
33. 11149
34. 10109
35. 10108
36. 40844
37. 191
38. 41027
39. 13563
40. 10888
41. 10644
42. 40932
43. 40503
44. 10645
45. 40847
46. 182
47. 11403
48. 10683
49. 550
50. 10640
51. 11238
52. 40881
53. 12651
54. 168
55. 11166
56. 181
57. 40854
58. 40505
59. 11039
60. 10657
61. 355
62. 40100
63. 10290
64. 199
65. 41549
66. 10101
67. 11369
68. 10842
69. 13393
70. 12113
71. 41775
72. 10258
73. 11146
74. 148
75. 13209
76. 42495
77. 41412
78. 12076
79. 40888
80. 41996
81. 11151
82. 12114
83. 10663
84. 10887
85. 10110
86. 10058
87. 11150
88. 10272
89. 11123
90. 40797
91. 13132
92. 41885
93. 40812
94. 10160
95. 11294
96. 10203
97. 41302
98. 40008
99. 10818
100. 10824
101. 11079
102. 43071
10

In [135]:
import requests
import gender_guesser.detector as gender

detector = gender.Detector()

# Cache to store gender of names
gender_cache = {}

def predict_gender_first_name_api(name):
    # Check if name's gender is already in cache
    if name in gender_cache:
        return gender_cache[name]
    
    api_url = 'https://api.genderize.io?name=' + name
    response = requests.get(api_url)
    # Check if the request was successful (200 OK)
    if response.status_code == 200:
        data = response.json()  # Parse the response content as JSON
        try:
            gender = data["gender"].capitalize()
            if data["gender"] in {'male', 'female'}:
                # Save the gender to cache before returning
                gender_cache[name] = data["gender"]
                return data["gender"]
            else:
                # Save 'Unknown' to cache before returning
                gender_cache[name] = "Unknown"
                return "Unknown"
            
        except Exception as e:
            print(f"Error predicting gender for {name}: {e}")
            return "Unknown"
    else:
        print(f"Request failed. Status code: {response.status_code}")
        return "Unknown"

# Function to get the first name from a full name
def get_first_name(full_name):
    if full_name and full_name.strip():  # Check if full_name is not empty and not just whitespace
        return full_name.split()[0]
    else:
        return "None"

# Function to predict gender for the first name
def predict_gender_first_name(name):
    gender = detector.get_gender(name)
    if name == "None":
        return "None"
    elif gender == "unknown":
        gender_api = predict_gender_first_name_api(name)
        return gender_api
    else:
        return gender

# Apply gender prediction to the first name in each author column and create new gender columns
all_data['Gender_Author 1'] = all_data['Author 1'].apply(get_first_name).apply(predict_gender_first_name)
all_data['Gender_Author 2'] = all_data['Author 2'].apply(get_first_name).apply(predict_gender_first_name)
all_data['Gender_Author 3'] = all_data['Author 3'].apply(get_first_name).apply(predict_gender_first_name)

# Map the gender predictions to more readable form
all_data['Gender_Author 1'] = all_data['Gender_Author 1'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
all_data['Gender_Author 2'] = all_data['Gender_Author 2'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
all_data['Gender_Author 3'] = all_data['Gender_Author 3'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})

print(all_data)


Error predicting gender for Sang-Hyop: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Chunbei: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for F.: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Meng-Na: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Marwân-al-Qays: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for E.: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Pierre–André: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Shao-Hsun: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Sevias: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for A.: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for Shasikanta: 'NoneType' object has no attribute 'capitalize'
Error predicting gender for M.J.: 'NoneType' object has no att

KeyboardInterrupt: 

In [157]:
# Function to get the article gender based on author genders
def get_article_gender(row):
    # Extract genders of authors, ignoring 'Unknown'
    genders = [row[f'Gender_Author {i+1}'] for i in range(3) if row[f'Gender_Author {i+1}'] != 'Unknown']
    
    # If there are no known gender values
    if len(genders) == 0:
        return 'Unknown'
    # If all known gender values are 'Male'
    elif all(gender == 'Male' for gender in genders):
        return 'Male'
    # If all known gender values are 'Female'
    elif all(gender == 'Female' for gender in genders):
        return 'Female'
    # If there's a mix of male and female among the known gender values
    else:
        return 'Mix'

# Apply the function to create the "Article_Gender" column
all_data['Article_Gender'] = all_data.apply(get_article_gender, axis=1)

# Display the updated DataFrame
print(all_data)


       Volume  Issue                          Journal Name    Published Date  \
0         1.0    1.0  Review of Economics of the Household              None   
1         1.0    1.0  Review of Economics of the Household              None   
2         1.0    1.0  Review of Economics of the Household              None   
3         1.0    1.0  Review of Economics of the Household              None   
4         1.0    1.0  Review of Economics of the Household              None   
...       ...    ...                                   ...               ...   
80001    57.0    6.0                        Intereconomics  16 December 2022   
80002    57.0    6.0                        Intereconomics  16 December 2022   
80003    57.0    6.0                        Intereconomics  16 December 2022   
80004    57.0    6.0                        Intereconomics  16 December 2022   
80005    57.0    6.0                        Intereconomics  16 December 2022   

                                       

In [158]:
all_data.to_csv('test.csv', index=False)

In [162]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from concurrent.futures import ThreadPoolExecutor
import datetime
import os

counter = [0]

def scrape_intro_and_citations(row):
    global counter
    url = row[all_data.columns.tolist().index('Link') + 1]
    
    if not url:  # Check if the URL is None or empty
        return (row[0], None, None)
    
    response = requests.get(url)
    time.sleep(1)  # Sleep for 1 second after each request
    
    # Increment the counter and print if it's a multiple of 100
    counter[0] += 1
    if counter[0] % 100 == 0:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{timestamp}: {journal_name} {counter[0]}")
    
    if response.status_code == 200:
        webpage = response.text
        soup = BeautifulSoup(webpage, 'html.parser')
        
        # Scrape the introduction
        intro_div = soup.find('div', class_='c-article-section__content', id='Sec1-content')
        intro = ' '.join(p.get_text() for p in intro_div.find_all('p')) if intro_div else None
        
        # Scrape the number of citations
        citations_tag = soup.find('span', string='Citations')
        citations = citations_tag.find_parent('p', class_='c-article-metrics-bar__count').get_text().split()[0] if citations_tag else None
        
        return (row[0], intro, citations)
    elif response.status_code == 429:  # Rate limit error
        time.sleep(300)
        return scrape_intro_and_citations(row)
    return (row[0], None, None)

path_to_save = "/Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer/Journals_W_Intro"

for journal_name in all_data['Journal Name'].unique():
    file_path = f"{path_to_save}/{journal_name}.csv"
    
    # Check if the file already exists
    if os.path.exists(file_path):
        print(f"Journal {journal_name} already processed. Skipping...")
        continue

    counter[0] = 0  # Reset counter for each journal
    print(f"Processing journal: {journal_name}")
    
    # Get all rows for the particular journal
    journal_data = all_data[all_data['Journal Name'] == journal_name].copy()
    
    # Filter only the rows with 'Female' or 'Male' for scraping
    scrape_data = journal_data[journal_data['Article_Gender'].isin(['Female', 'Male'])]

    # Use ThreadPoolExecutor to scrape data concurrently
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(scrape_intro_and_citations, scrape_data.itertuples(index=True, name=None)))

    # Update the journal_data DataFrame directly with the scraped values
    for index, intro, citations in results:
        journal_data.at[index, 'Intro'] = intro
        journal_data.at[index, 'Citations'] = citations

    # Save the entire journal_data DataFrame (including rows with 'Mix' and 'Unknown') to a CSV
    journal_data.to_csv(file_path, index=False)

    print(f"Saved data for journal {journal_name} to {file_path}")


Journal Review of Economics of the Household already processed. Skipping...
Journal Review of Economic Design already processed. Skipping...
Journal Asia-Pacific Journal of Regional Science already processed. Skipping...
Processing journal: Journal of Economic Growth
2023-08-09 14:27:22: Journal of Economic Growth 100
2023-08-09 14:28:04: Journal of Economic Growth 200
Saved data for journal Journal of Economic Growth to /Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer/Journals_W_Intro/Journal of Economic Growth.csv
Processing journal: Review of Evolutionary Political Economy
Saved data for journal Review of Evolutionary Political Economy to /Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer/Journals_W_Intro/Review of Evolutionary Political Economy.csv
Processing journal: Journal of Economic Interaction and Coordination
2023-08-09 14:29:49: Journal of Economic Interaction and Coordination 100
2023-08-09 14:30:38: Journal of Economic Interaction and

KeyboardInterrupt: 

In [153]:
print(all_data)

       Volume  Issue                          Journal Name    Published Date  \
0         1.0    1.0  Review of Economics of the Household              None   
1         1.0    1.0  Review of Economics of the Household              None   
2         1.0    1.0  Review of Economics of the Household              None   
3         1.0    1.0  Review of Economics of the Household              None   
4         1.0    1.0  Review of Economics of the Household              None   
...       ...    ...                                   ...               ...   
80001    57.0    6.0                        Intereconomics  16 December 2022   
80002    57.0    6.0                        Intereconomics  16 December 2022   
80003    57.0    6.0                        Intereconomics  16 December 2022   
80004    57.0    6.0                        Intereconomics  16 December 2022   
80005    57.0    6.0                        Intereconomics  16 December 2022   

                                       

In [148]:
# Count non-NaN values for 'Link' column
link_count = all_data['Intro'].notna().sum()

# Count non-NaN values for 'Citations' column
citations_count = all_data['Citations'].notna().sum()

print("Number of non-NaN values in Intro column:", link_count)
print("Number of non-NaN values in Citations column:", citations_count)


Number of non-NaN values in Intro column: 0
Number of non-NaN values in Citations column: 0


In [149]:
def test_scrape(url):
    response = requests.get(url)
    if response.status_code == 200:
        webpage = response.text
        soup = BeautifulSoup(webpage, 'html.parser')
        
        # Scrape the introduction
        intro_div = soup.find('div', class_='c-article-section__content', id='Sec1-content')
        intro = ' '.join(p.get_text() for p in intro_div.find_all('p')) if intro_div else None
        
        # Scrape the number of citations
        citations_tag = soup.find('span', string='Citations')
        citations = citations_tag.find_parent('p', class_='c-article-metrics-bar__count').get_text().split()[0] if citations_tag else None
        
        print("Intro:")
        print(intro)
        print("\nCitations:")
        print(citations)
    else:
        print("Failed to fetch the webpage")

# Test URL
test_url = "https://link.springer.com/article/10.1007/s10887-022-09210-8"  # replace with one of your known URLs
test_scrape(test_url)


Intro:
History abounds with episodes of forced migration triggered by civil unrest, wars, natural disasters, and state mandated expulsions or relocations. If current times are any indication, we will continue to witness similar events going forward. Forced migration may yield qualitatively different consequences for stayers, migrants, and destination regions. Yet, the literature so far has largely focused on the impact of forced migration on receiving populations (e.g., Hornung, (2014)) or migrants themselves (e.g., Becker et al., (2020)), while paying less attention to the long-term prosperity of sending regions.Footnote 1 How sending regions fare in the long run depends on both the adverse effects of forced out-migrations and how departing populations had shaped the development potential of local economies until they left. These latter effects—the economic legacy of émigrés—may operate through human capital spillovers from departing groups onto the staying population and the producti

In [76]:
# Iterate through the DataFrame and set "Intro_check" based on the presence of "Intro"
df['Intro_check'] = df['Intro'].notnull().astype(int)

# Display the updated DataFrame
print(df)

     Volume  Issue                                               Link  \
0         1      1  https://link.springer.com/article/10.1007/BF00...   
1         1      1  https://link.springer.com/article/10.1007/BF00...   
2         1      1  https://link.springer.com/article/10.1007/BF00...   
3         1      1  https://link.springer.com/article/10.1007/BF00...   
4         1      1  https://link.springer.com/article/10.1007/BF00...   
..      ...    ...                                                ...   
357      27      3  https://link.springer.com/article/10.1007/s108...   
358      27      3  https://link.springer.com/article/10.1007/s108...   
359      27      4  https://link.springer.com/article/10.1007/s108...   
360      27      4  https://link.springer.com/article/10.1007/s108...   
361      27      4  https://link.springer.com/article/10.1007/s108...   

                                                 Title           Author 1  \
0                                 Democracy an

In [77]:
import os
import pandas as pd

# Assuming 'df' is your DataFrame
# For example:
# df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': ['A', 'B', 'C']})

# Get the current working directory
current_directory = os.getcwd()

# Define the file name you want to save
file_name = 'Springer_dataframe.csv'

# Join the current directory with the file name
file_path = os.path.join(current_directory, file_name)

# Save the DataFrame to CSV
df.to_csv(file_path, index=False)

print(f"DataFrame has been saved to '{file_path}'.")


DataFrame has been saved to '/Users/Magnus/Documents/GitHub/SkoderIDinDrink/Exam_project/Springer_dataframe.csv'.
